# Appendix A: Data Cleaning

## Prepare Data

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from sklearn.preprocessing import LabelEncoder

In [44]:
# Load data: CPS micro data from IPUMS for November 2006, 2011, 2016
data = pd.read_csv('IPUMS_CPS_181104.csv')

# Check data importation
data.head()

,YEAR,SERIAL,MONTH,HWTFINL,CPSID,REGION,STATEFIP,METRO,COUNTY,FAMINC,...,OCC,IND,CLASSWKR,DURUNEMP,WHYUNEMP,WKSTAT,NUMJOB,EDUC,SCHLCOLL,DIFFANY
0,2006,1,11,2410.8311,20050802654600,32,1,1,1003,600,...,5820,6990,22,999,0,11,0,73,0,NaN
1,2006,1,11,2410.8311,20050802654600,32,1,1,1003,600,...,0,0,0,999,0,99,0,1,0,NaN
2,2006,2,11,3253.7650,20050902383200,32,1,1,1003,500,...,730,7070,22,999,0,11,0,81,0,NaN
3,2006,3,11,2754.1651,20051006048500,32,1,1,1003,996,...,0,0,0,999,0,99,0,125,0,NaN
4,2006,3,11,2754.1651,20051006048500,32,1,1,1003,996,...,0,0,0,999,0,99,0,91,0,NaN


## Clean Data

Variable: AGE

In [45]:
# In our analysis, we want to look at the prime-age workforce, so we will restrict the dataset to ages 25-54
data = data[data['AGE']>=25]
data = data[data['AGE']<55]

# Check the distribution of age
data.AGE.describe()

count    161420.000000
mean         39.869576
std           8.666890
min          25.000000
25%          32.000000
50%          40.000000
75%          47.000000
max          54.000000
Name: AGE, dtype: float64

Variable: EMPSTAT (Employment Status)

In [46]:
# Create a new variable EMPSTAT2
data['EMPSTAT2'] = data['EMPSTAT']

# At this point in our analysis, we want to look at the lables for each employment status code.

# Recode 10 (employed at work) and 12 (has job, not at work last week) as Employed
data.loc[data.EMPSTAT == 10,'EMPSTAT2'] = 'Employed'
data.loc[data.EMPSTAT == 12,'EMPSTAT2'] = 'Employed'

# Recode cateogries 20 (Unemployed), 21 (Unemployed, experienced worker), and 22 (Unemployed, new worker) as Unemployed
data.loc[data.EMPSTAT == 20,'EMPSTAT2'] = 'Unemployed'
data.loc[data.EMPSTAT == 21,'EMPSTAT2'] = 'Unemployed'
data.loc[data.EMPSTAT == 22,'EMPSTAT2'] = 'Unemployed'

# Recode the remaining codes in EMPSTAT2 with respective labels
data.loc[data.EMPSTAT == 1,'EMPSTAT2'] = 'Armed Forces'
data.loc[data.EMPSTAT == 31,'EMPSTAT2'] = 'Not In Labor Force, housework'
data.loc[data.EMPSTAT == 32,'EMPSTAT2'] = 'Not In Labor Force, unable to work'
data.loc[data.EMPSTAT == 33,'EMPSTAT2'] = 'Not In Labor Force, school'
data.loc[data.EMPSTAT == 34,'EMPSTAT2'] = 'Not In Labor Force, other'
data.loc[data.EMPSTAT == 35,'EMPSTAT2'] = 'Not In Labor Force, unpaid, less than 15 hours'
data.loc[data.EMPSTAT == 36,'EMPSTAT2'] = 'Not In Labor Force, retired'

# Verify EMPSTAT values
data.EMPSTAT2.value_counts()

Employed                              126355
Not In Labor Force, other              18316
Not In Labor Force, unable to work      7809
Unemployed                              6075
Not In Labor Force, retired             1777
Armed Forces                            1088
Name: EMPSTAT2, dtype: int64

Variable: DURUNEMP (Continuous week employed)

In [47]:
# Create a new variable DURUNEMP2 
data['DURUNEMP2'] = data['DURUNEMP']

# Recode 999 (missing) as NaN
data.loc[data['DURUNEMP2']== 999,'DURUNEMP2']=np.nan

# Check the distribution of DURUNEMP2
data.DURUNEMP2.describe()

count    6075.000000
mean       28.944691
std        33.965344
min         0.000000
25%         4.000000
50%        13.000000
75%        43.000000
max       119.000000
Name: DURUNEMP2, dtype: float64

Variable: CLASSWKR (Class of Worker)

In [48]:
# Create a new variable CLASSWKR2 
data['CLASSWKR2'] = data['CLASSWKR']

# Recode 0 (NIU) as NIU
data.loc[data.CLASSWKR == 0,'CLASSWKR2'] = 'NIU'

# Recode 13 (Self-Employed, non incorporated) and 14 (Self-Employed, incorporated) as (Self-Employed)
data.loc[data.CLASSWKR == 13,'CLASSWKR2'] = 'Self-Employed'
data.loc[data.CLASSWKR == 14,'CLASSWKR2'] = 'Self-Employed'

# Recode 21-29 as 20 (Works for wages or salary)
data.loc[data.CLASSWKR == 21,'CLASSWKR2'] = 'Employed'
data.loc[data.CLASSWKR == 22,'CLASSWKR2'] = 'Employed'
data.loc[data.CLASSWKR == 23,'CLASSWKR2'] = 'Employed'
data.loc[data.CLASSWKR == 24,'CLASSWKR2'] = 'Employed'
data.loc[data.CLASSWKR == 25,'CLASSWKR2'] = 'Employed'
data.loc[data.CLASSWKR == 26,'CLASSWKR2'] = 'Employed'
data.loc[data.CLASSWKR == 27,'CLASSWKR2'] = 'Employed'
data.loc[data.CLASSWKR == 28,'CLASSWKR2'] = 'Employed'
data.loc[data.CLASSWKR == 29,'CLASSWKR2'] = 'Employed'

# Recode 99 (Missing/Unknown) as missing
data.loc[data['CLASSWKR2']== 99,'CLASSWKR2']=np.nan

# Verify CLASSWKR2 values
data.CLASSWKR2.value_counts()

Employed         120849
NIU               27223
Self-Employed     13348
Name: CLASSWKR2, dtype: int64

Variable: REGION

In [49]:
# Recode REGION with region labels
data.loc[data.REGION == 11, 'REGION2'] = "New England Division"
data.loc[data.REGION == 12, 'REGION2'] = "Middle Atlantic Division"
data.loc[data.REGION == 21, 'REGION2'] = "East North Central Division"
data.loc[data.REGION == 22, 'REGION2'] = "West North Central Division"
data.loc[data.REGION == 31, 'REGION2'] = "South Atlantic Division"
data.loc[data.REGION == 32, 'REGION2'] = "East South Central Division"
data.loc[data.REGION == 33, 'REGION2'] = "West South Central Division"
data.loc[data.REGION == 41, 'REGION2'] = "Mountain Division"
data.loc[data.REGION == 42, 'REGION2'] = "Pacific Division"
data.loc[data.REGION == 97, 'REGION2'] = "State not identified"

# Verify REGION values
data['REGION2'].value_counts()

South Atlantic Division        29733
Pacific Division               24093
East North Central Division    18139
Mountain Division              18117
West North Central Division    17360
New England Division           15866
West South Central Division    14831
Middle Atlantic Division       14617
East South Central Division     8664
Name: REGION2, dtype: int64

Variable: RACE

In [50]:
# Recode RACE with race labels
data.loc[data.RACE == 100, 'RACE2'] = "White"
data.loc[data.RACE == 200, 'RACE2'] = "Black"
data.loc[data.RACE == 300, 'RACE2'] = "Indian American"
data.loc[data.RACE == 651, 'RACE2'] = "Asian"
data.loc[data.RACE == 652, 'RACE2'] = "Hawaiian"

data.loc[data.RACE == 801, 'RACE2'] = "Multirace"
data.loc[data.RACE == 802, 'RACE2'] = "Multirace"
data.loc[data.RACE == 803, 'RACE2'] = "Multirace"
data.loc[data.RACE == 804, 'RACE2'] = "Multirace"
data.loc[data.RACE == 805, 'RACE2'] = "Multirace"
data.loc[data.RACE == 806, 'RACE2'] = "Multirace"
data.loc[data.RACE == 807, 'RACE2'] = "Multirace"
data.loc[data.RACE == 808, 'RACE2'] = "Multirace"
data.loc[data.RACE == 809, 'RACE2'] = "Multirace"
data.loc[data.RACE == 810, 'RACE2'] = "Multirace"
data.loc[data.RACE == 811, 'RACE2'] = "Multirace"
data.loc[data.RACE == 812, 'RACE2'] = "Multirace"
data.loc[data.RACE == 813, 'RACE2'] = "Multirace"
data.loc[data.RACE == 814, 'RACE2'] = "Multirace"
data.loc[data.RACE == 815, 'RACE2'] = "Multirace"
data.loc[data.RACE == 816, 'RACE2'] = "Multirace"
data.loc[data.RACE == 817, 'RACE2'] = "Multirace"
data.loc[data.RACE == 818, 'RACE2'] = "Multirace"
data.loc[data.RACE == 819, 'RACE2'] = "Multirace"
data.loc[data.RACE == 820, 'RACE2'] = "Multirace"
data.loc[data.RACE == 830, 'RACE2'] = "Multirace"

# Verify MARST2 values
data['RACE2'].value_counts()

White              130646
Black               16597
Asian                8717
Multirace            2746
Indian American      1970
Hawaiian              744
Name: RACE2, dtype: int64

Variable: Sex

In [51]:
# Recode SEX with labels
data.loc[data.SEX == 1, 'SEX2'] = "Male"
data.loc[data.SEX == 2, 'SEX2'] = "Female"
data.loc[data.SEX == 9, 'SEX2'] = "NIU"

# Verify SEX2 values
data['SEX2'].value_counts()

Female    83052
Male      78368
Name: SEX2, dtype: int64

Variable: MARST (Marital Status)

In [52]:
# Recode MARST with labels
data.loc[data.MARST == 1, 'MARST2'] = "Married"
data.loc[data.MARST == 2, 'MARST2'] = "Married"
data.loc[data.MARST == 3, 'MARST2'] = "Seprated, divorced, or widowed"
data.loc[data.MARST == 4, 'MARST2'] = "Seprated, divorced, or widowed"
data.loc[data.MARST == 5, 'MARST2'] = "Seprated, divorced, or widowed"
data.loc[data.MARST == 6, 'MARST2'] = "Single"
data.loc[data.MARST == 9, 'MARST2'] = "NIU"

# Verify MARST2 values
data['MARST2'].value_counts()

Married                           98866
Single                            38439
Seprated, divorced, or widowed    24115
Name: MARST2, dtype: int64

Variable: YRIMMIG (Year of Immigration)

In [53]:
# Recode YRIMMIG with label
data.loc[data.YRIMMIG > 0, 'YRIMMIG2'] = data.YRIMMIG

# Recode missing data
data.loc[data.YRIMMIG == 0, 'YRIMMIG2'] = 'NIU'

# Verify YRIMMIG2 values
data['YRIMMIG2'].value_counts()

NIU       132886
2001.0      2603
1999.0      2116
1991.0      1786
1997.0      1717
1995.0      1692
2003.0      1691
1979.0      1520
1989.0      1477
1993.0      1261
2005.0      1234
1981.0      1218
1985.0      1161
1987.0      1136
1974.0      1065
2008.0      1009
1983.0       884
2010.0       820
2016.0       792
2006.0       749
1969.0       610
2013.0       492
2009.0       476
2011.0       460
1964.0       376
1959.0       188
1949.0         1
Name: YRIMMIG2, dtype: int64

Variable: CITIZEN

In [54]:
# Recode CITIZEN with label
data.loc[data.CITIZEN == 1, 'CITIZEN2'] = "Born in US"
data.loc[data.CITIZEN == 2, 'CITIZEN2'] = "Born in US outlying"
data.loc[data.CITIZEN == 3, 'CITIZEN2'] = "Born acroad of American parents"
data.loc[data.CITIZEN == 4, 'CITIZEN2'] = "Naturalized citizen"
data.loc[data.CITIZEN == 5, 'CITIZEN2'] = "Not a citizen"
data.loc[data.CITIZEN == 9, 'CITIZEN2'] = "NIU"

# Verify CITIZEN2 values
data['CITIZEN2'].value_counts()

Born in US                         132886
Not a citizen                       15619
Naturalized citizen                 10285
Born acroad of American parents      1681
Born in US outlying                   949
Name: CITIZEN2, dtype: int64

Variable: LABFORCE (Labor Force Status)

In [55]:
# Recode LABFORCE with label
data.loc[data.LABFORCE == 1, 'LABFORCE2'] = "No, not in the labor force"
data.loc[data.LABFORCE == 2, 'LABFORCE2'] = "Yes, in the labor force"
data.loc[data.LABFORCE == 0, 'LABFORCE2'] = "NIU"

# Verify LABFORCE2 values
data['LABFORCE2'].value_counts()

Yes, in the labor force       132430
No, not in the labor force     27902
NIU                             1088
Name: LABFORCE2, dtype: int64

Variable: WKSTAT (Full or Part Time Status)

In [56]:
# Create a new variable WKSTAT2 
data['WKSTAT2'] = data['WKSTAT']

# Recode data codes with labels
data.loc[data.WKSTAT == 11,'WKSTAT2'] = 'Full-time hours (35+), usually full-time'
data.loc[data.WKSTAT == 12,'WKSTAT2'] = 'Part-time for non-economic reasons, usually full-time'
data.loc[data.WKSTAT == 13,'WKSTAT2'] = 'Not at work, usually full-time'
data.loc[data.WKSTAT == 14,'WKSTAT2'] = 'Full-time hours, usually part-time for economic reasons'
data.loc[data.WKSTAT == 15,'WKSTAT2'] = 'Full-time hours, usually part-time for non-economic reasons'
data.loc[data.WKSTAT == 21,'WKSTAT2'] = 'Part-time for economic reasons, usually full-time'
data.loc[data.WKSTAT == 22,'WKSTAT2'] = 'Part-time hours, usually part-time for economic reasons'
data.loc[data.WKSTAT == 41,'WKSTAT2'] = 'Part-time hours, usually part-time for non-economic reasons'
data.loc[data.WKSTAT == 42,'WKSTAT2'] = 'Not at work, usually part-time'
data.loc[data.WKSTAT == 50,'WKSTAT2'] = 'Unemployed, seeking full-time work'
data.loc[data.WKSTAT == 60,'WKSTAT2'] = 'Unemployed, seeking part-time work'

# Set value = 99 as NaN
data.loc[data['WKSTAT']== 99,'WKSTAT2']=np.nan

# Verify WHYUNEMP2 Values
data['WKSTAT2'].value_counts()

Full-time hours (35+), usually full-time                       95572
Part-time for non-economic reasons, usually full-time          11445
Part-time hours, usually part-time for non-economic reasons    11258
Unemployed, seeking full-time work                              5555
Part-time hours, usually part-time for economic reasons         3205
Not at work, usually full-time                                  2351
Part-time for economic reasons, usually full-time               1210
Not at work, usually part-time                                   696
Unemployed, seeking part-time work                               520
Full-time hours, usually part-time for non-economic reasons      449
Full-time hours, usually part-time for economic reasons          169
Name: WKSTAT2, dtype: int64

Variable: WHYUNEMP (Reason for Unemployment)

In [57]:
# Recode data codes with labels
data.loc[data.WHYUNEMP == 0,'WHYUNEMP2'] = 'NIU'
data.loc[data.WHYUNEMP == 1,'WHYUNEMP2'] = 'Job loser / on layoff'
data.loc[data.WHYUNEMP == 2,'WHYUNEMP2'] = 'Other job loser'
data.loc[data.WHYUNEMP == 3,'WHYUNEMP2'] = 'Temporary job ended'
data.loc[data.WHYUNEMP == 4,'WHYUNEMP2'] = 'Job leaver'
data.loc[data.WHYUNEMP == 5,'WHYUNEMP2'] = 'Re-entrant'
data.loc[data.WHYUNEMP == 6,'WHYUNEMP2'] = 'New-entrant'

# Verify WHYUNEMP2 Values
data['WHYUNEMP2'].value_counts()

NIU                      155345
Other job loser            2257
Re-entrant                 1581
Temporary job ended         713
Job loser / on layoff       703
Job leaver                  676
New-entrant                 145
Name: WHYUNEMP2, dtype: int64

Variable: EDUC (Educational Attainment)

In [58]:
# Create a new variable EDUC2 
data['EDUC2'] = data['EDUC']

# Recode everything less than 72 as 1, signifying less than a high school diploma
data.loc[data.EDUC <72,'EDUC2'] = 'Less than High School'

# Recode 72 and 73 as 2, signifying a high school diploma or equivalent
data.loc[data.EDUC== 72,'EDUC2'] = 'High School Diploma or Equivalent'
data.loc[data.EDUC== 73,'EDUC2'] = 'High School Diploma or Equivalent'

# Recode 80 and 81 as 3, signifying some college but no degree
data.loc[data.EDUC== 80,'EDUC2'] = 'Some College but No Degree'
data.loc[data.EDUC== 81,'EDUC2'] = 'Some College but No Degree'

# Recode 90, 91, and 92 as 4, signifying completing an Associate's degree
data.loc[data.EDUC== 90,'EDUC2'] = 'Associates Degree'
data.loc[data.EDUC== 91,'EDUC2'] = 'Associates Degree'
data.loc[data.EDUC== 92,'EDUC2'] = 'Associates Degree'

# Recode 110 and 111 as 5, signifying completing a Bachelor's degree
data.loc[data.EDUC== 110,'EDUC2'] = 'Bachelors Degree'
data.loc[data.EDUC== 111,'EDUC2'] = 'Bachelors Degree'

# Recode > 111 as 6, signifying completing a Graduate Program
data.loc[data.EDUC > 111,'EDUC2'] = 'Graduate Degree'

# Set value = 999 as NaN
data.loc[data['EDUC2']== 999,'EDUC2']=np.nan

# Verify EDUC2 Values
data['EDUC2'].value_counts()

High School Diploma or Equivalent    46189
Bachelors Degree                     36033
Some College but No Degree           27800
Graduate Degree                      18289
Associates Degree                    17345
Less than High School                15764
Name: EDUC2, dtype: int64

Variable: STATEFIPS (State FIPS Code)

In [59]:
# Create a new variable SCHLCOLL2 
data['STATEFIP2'] = data['STATEFIP']

#Recode data codes with labels
data.loc[data.STATEFIP == 1,'STATEFIP2'] = 'Alabama'
data.loc[data.STATEFIP == 2,'STATEFIP2'] = 'Alaska'
data.loc[data.STATEFIP == 4,'STATEFIP2'] = 'Arizona'
data.loc[data.STATEFIP == 5,'STATEFIP2'] = 'Arkansas'
data.loc[data.STATEFIP == 6,'STATEFIP2'] = 'California'
data.loc[data.STATEFIP == 8,'STATEFIP2'] = 'Colorado'
data.loc[data.STATEFIP == 9,'STATEFIP2'] = 'Connecticut'

data.loc[data.STATEFIP == 10,'STATEFIP2'] = 'Delaware'
data.loc[data.STATEFIP == 11,'STATEFIP2'] = 'District of Columbia'
data.loc[data.STATEFIP == 12,'STATEFIP2'] = 'Florida'
data.loc[data.STATEFIP == 13,'STATEFIP2'] = 'Goergia'
data.loc[data.STATEFIP == 15,'STATEFIP2'] = 'Hawaii'
data.loc[data.STATEFIP == 16,'STATEFIP2'] = 'Idaho'
data.loc[data.STATEFIP == 17,'STATEFIP2'] = 'Illinois'
data.loc[data.STATEFIP == 18,'STATEFIP2'] = 'Indiana'
data.loc[data.STATEFIP == 19,'STATEFIP2'] = 'Iowa'

data.loc[data.STATEFIP == 20,'STATEFIP2'] = 'Kansas'
data.loc[data.STATEFIP == 21,'STATEFIP2'] = 'Kentucky'
data.loc[data.STATEFIP == 22,'STATEFIP2'] = 'Louisiana'
data.loc[data.STATEFIP == 23,'STATEFIP2'] = 'Maine'
data.loc[data.STATEFIP == 24,'STATEFIP2'] = 'Maryland'
data.loc[data.STATEFIP == 25,'STATEFIP2'] = 'Massachusetts'
data.loc[data.STATEFIP == 26,'STATEFIP2'] = 'Michigan'
data.loc[data.STATEFIP == 27,'STATEFIP2'] = 'Minnesota'
data.loc[data.STATEFIP == 28,'STATEFIP2'] = 'Mississippi'
data.loc[data.STATEFIP == 29,'STATEFIP2'] = 'Missouri'

data.loc[data.STATEFIP == 30,'STATEFIP2'] = 'Montana'
data.loc[data.STATEFIP == 31,'STATEFIP2'] = 'Nebraska'
data.loc[data.STATEFIP == 32,'STATEFIP2'] = 'Nevada'
data.loc[data.STATEFIP == 33,'STATEFIP2'] = 'New Hampshire'
data.loc[data.STATEFIP == 34,'STATEFIP2'] = 'New Jersey'
data.loc[data.STATEFIP == 35,'STATEFIP2'] = 'New Mexico'
data.loc[data.STATEFIP == 36,'STATEFIP2'] = 'New York'
data.loc[data.STATEFIP == 37,'STATEFIP2'] = 'North Carolina'
data.loc[data.STATEFIP == 38,'STATEFIP2'] = 'North Dakota'
data.loc[data.STATEFIP == 39,'STATEFIP2'] = 'Ohio'

data.loc[data.STATEFIP == 40,'STATEFIP2'] = 'Oklahoma'
data.loc[data.STATEFIP == 41,'STATEFIP2'] = 'Oregon'
data.loc[data.STATEFIP == 42,'STATEFIP2'] = 'Pennsylvania'
data.loc[data.STATEFIP == 44,'STATEFIP2'] = 'Rhode Island'
data.loc[data.STATEFIP == 45,'STATEFIP2'] = 'South Carolina'
data.loc[data.STATEFIP == 46,'STATEFIP2'] = 'South Dakota'
data.loc[data.STATEFIP == 47,'STATEFIP2'] = 'Tennessee'
data.loc[data.STATEFIP == 48,'STATEFIP2'] = 'Texas'
data.loc[data.STATEFIP == 49,'STATEFIP2'] = 'Utah'

data.loc[data.STATEFIP == 50,'STATEFIP2'] = 'Vermont'
data.loc[data.STATEFIP == 51,'STATEFIP2'] = 'Virginia'
data.loc[data.STATEFIP == 53,'STATEFIP2'] = 'Washington'
data.loc[data.STATEFIP == 54,'STATEFIP2'] = 'West Virginia'
data.loc[data.STATEFIP == 55,'STATEFIP2'] = 'Wisconsin'
data.loc[data.STATEFIP == 56,'STATEFIP2'] = 'Wyoming'

# Verify STATEFIP2 Values
data.STATEFIP2.value_counts()

California              14217
Texas                    8711
New York                 6700
Florida                  6627
Illinois                 4960
Pennsylvania             4516
Ohio                     4238
Michigan                 3723
Goergia                  3648
Virginia                 3447
New Jersey               3401
Minnesota                3354
Maryland                 3349
North Carolina           3303
Colorado                 3224
New Hampshire            3059
Connecticut              3055
Washington               2891
Massachusetts            2776
Wisconsin                2692
District of Columbia     2663
Iowa                     2616
Missouri                 2596
Indiana                  2526
Nevada                   2498
Hawaii                   2426
Tennessee                2399
Rhode Island             2385
Maine                    2368
Oregon                   2357
West Virginia            2296
Nebraska                 2294
Delaware                 2266
Arizona   

Variable: NUMJOB (How many jobs had in past week)

In [60]:
# Create a new variable WKSTAT2 
data['NUMJOB2'] = data['NUMJOB']

# Recode data codes with labels
data.loc[data.NUMJOB == 0,'NUMJOB2'] = 'NIU'
data.loc[data.NUMJOB == 2,'NUMJOB2'] = '3 Jobs'
data.loc[data.NUMJOB == 3,'NUMJOB2'] = '3 Jobs'
data.loc[data.NUMJOB == 4,'NUMJOB2'] = '4 or more jobs'

# Verify NUMJOB2 Values
data.NUMJOB2.value_counts()

NIU               153777
3 Jobs              7551
4 or more jobs        92
Name: NUMJOB2, dtype: int64

Variable: SCHLCOLL (School or college attendance)

In [61]:
# Create a new variable SCHLCOLL2 
data['SCHLCOLL2'] = data['SCHLCOLL']

#Recode data codes with labels
data.loc[data.SCHLCOLL == 0,'SCHLCOLL2'] = 'NIU'
data.loc[data.SCHLCOLL == 1,'SCHLCOLL2'] = 'High school full time'
data.loc[data.SCHLCOLL == 2,'SCHLCOLL2'] = 'High school part time'
data.loc[data.SCHLCOLL == 3,'SCHLCOLL2'] = 'College or university full time'
data.loc[data.SCHLCOLL == 4,'SCHLCOLL2'] = 'College or university part time'
data.loc[data.SCHLCOLL == 5,'SCHLCOLL2'] = 'Does not attend school, college or university'

# Verify SCHLCOLL2 Values
data.SCHLCOLL2.value_counts()

NIU                                              111090
Does not attend school, college or university     47799
College or university full time                    1429
College or university part time                    1020
High school full time                                62
High school part time                                20
Name: SCHLCOLL2, dtype: int64

Variable: METRO (Metropolitan central city status)

In [62]:
# Recode data codes with labels
data.loc[data.METRO == 0,'METRO2'] = 'Not identifiable'
data.loc[data.METRO == 1,'METRO2'] = 'Not in metro area'
data.loc[data.METRO == 2,'METRO2'] = 'Central city'
data.loc[data.METRO == 3,'METRO2'] = 'Outside central city'
data.loc[data.METRO == 4,'METRO2'] = 'Central city status unknown'

# Set value = 999 as NaN
data.loc[data['METRO2']== 9,'METRO2']=np.nan

# Verify METRO2 Values
data.METRO2.value_counts()

Outside central city           62032
Central city                   40629
Not in metro area              29735
Central city status unknown    27671
Not identifiable                1353
Name: METRO2, dtype: int64

Variable: VETSTAT (Veteran Status)

In [63]:
# Recode data codes with labels
data.loc[data.VETSTAT == 0,'VETSTAT2'] = 'Not identifiable'
data.loc[data.VETSTAT == 1,'VETSTAT2'] = 'No service'
data.loc[data.VETSTAT == 2,'VETSTAT2'] = 'Yes'
data.loc[data.VETSTAT == 9,'VETSTAT2'] = 'Outside central cityUnknown/non-WWII'

# Verify VETSTAT2 Values
data.VETSTAT2.value_counts()

No service          151667
Yes                   8665
Not identifiable      1088
Name: VETSTAT2, dtype: int64

Variable: DIFFANY (Any disability)

In [64]:
# Recode data codes with labels
data.loc[data.DIFFANY == 0,'DIFFANY2'] = 'NIU'
data.loc[data.DIFFANY == 1,'DIFFANY2'] = 'No difficulty'
data.loc[data.DIFFANY == 2,'DIFFANY2'] = 'Has difficulty'
data.DIFFANY2 = data.DIFFANY2.fillna('NIU')

# Verify DIFFANY2 Values
data.DIFFANY2.value_counts()

No difficulty     96858
NIU               57873
Has difficulty     6689
Name: DIFFANY2, dtype: int64

Variable: IND (Industry)

In [65]:
# Create a new variable IND2 
data['IND2'] = data['IND']

# Recode 0 (N/A) and 9920 (Unemployed, last worked 5 years ago) as NIU
data.loc[data.IND == 0,'IND2'] = 'NIU'
data.loc[data.IND == 9920,'IND2'] = 'NIU'

# Recode Agriculture, Forestry, Fishing, and Hunting
agriculture = [170,180,190,270, 280, 290]
for x in agriculture:
    data.loc[data.IND == x,'IND2'] = 'Agriculture'

# Recode Mining, Quarrying, and Oil and Gas Extraction
mining = [370, 380, 390, 470,480, 490]
for x in mining:
    data.loc[data.IND == x,'IND2'] = 'Manufacturing'

# Construction
data.loc[data.IND == 770,'IND2'] = 'Construction'

# Manufacturing
manufacturing = [1070, 1080, 1090, 1170, 1180, 1190, 1270, 1280, 1290, 1370, 1390, 1470, 1480, 1490, 1570, 1590, 1670, 1680, 1690, 1770, 1790, 1870, 1880, 1890, 1990, 2070, 2090, 2170, 2180, 2190, 2270, 2280, 2290,2370,2380, 2390, 2470, 2480,2490,2570,2590,2670,2680,2690,2770,2780,2790,2870,2880,2890,2970,2980,2990,3070,3080,3090,3095, 3170,3180,3190,3290,3360,3365,3370,3380,3390,3470,3490,3570,3580,3590,3670,3680,3690,3890,3770,3780,3790,3870,3875,3890,3895,3960,3970,3980,3990]
for x in manufacturing:
    data.loc[data.IND == x,'IND2'] = 'Manufacturing'

# Wholesale Trade
wholesale = [4070,4080,4090,4170,4180,4190,4195,4260,4265,4270,4280,4290,4370,4380,4390,4470,4480,4490,4560,4570,4580,4585,4590]
for x in wholesale:
    data.loc[data.IND == x,'IND2'] = 'WholesaleTrade'
    
# Retail Trade
retail = [4670,4680,4690,4770,4780,4790,4795,4870,4880,4890,4970,4980,4990,5070,5080,5090,5170,5180,5190,5270,5275,5280,5290,5295,5370,5380,5390,5470,5480,5490,5570,5580,5590,5591,5592,5670,5680,5690,5790]
for x in retail:
    data.loc[data.IND == x,'IND2'] = 'RetailTrade'
    
# Transportation and Warehousing
trans = [6070,6080,6090,6170,6180,6190,6270,6280,6290,6370,6380,6390]
for x in trans:
    data.loc[data.IND == x,'IND2'] = 'TransportationWharehousing'
    
# Utilities
utilities = [570,580,590,670,680,690]
for x in utilities:
    data.loc[data.IND == x,'IND2'] = 'Utilities'
    
# Information and Communications
info = [6470,6480,6490,6570,6590,6670,6672,6675,6870,6680,6690,6692,6695,6770,6780]
for x in info:
    data.loc[data.IND == x,'IND2'] = 'InformationCommunications'

# Finance, Insurance, Real Estate, and Rental and Leasing
finance = [6870,6880,6890,6970,6990,7070,7080,7170,7180,7190]
for x in finance:
    data.loc[data.IND == x,'IND2'] = 'Finance'
    
# Professional, Scientific, Management, Administrative, and Waste Management Services
professional = [7270,7280,7290,7370,7380,7390,7460,7470,7480,7490,7570,7580,7590,7670,7680,7690,7770,7780,7790]
for x in professional:
    data.loc[data.IND == x,'IND2'] = 'ProfessinalServices'
    
# Educational, Health and Social Services
social = [7860,7870,7880,7890,7970,7980,7990,8070,8080,8090,8170,8180,8190,8270,8290,8370,8380,8390,8470]
for x in social:
    data.loc[data.IND == x,'IND2'] = 'SocialServices'
    
# Entertainment
entertainment = [8560,8570,8580,8590,8660,8670,8680,8690]
for x in entertainment:
    data.loc[data.IND == x,'IND2'] = 'Entertainment'
    
# Other Services
other = [8770,8780,8790,8870,8880,8890,8970,8980,8990,9070,9080,9090,9160,9170,9180,9190,9290]
for x in other:
    data.loc[data.IND == x,'IND2'] = 'Other'
    
# Public Administration
admin = [9370,9380,9390,9390,9470,9480,9490,9570,9590]
for x in admin:
    data.loc[data.IND == x,'IND2'] = 'PublicAdmin'
    
# Armed Force
army = [9670,9680,9690,9770,9780,9790,9870,9890]
for x in army:
    data.loc[data.IND == x,'IND2'] = 'ArmedForces'

# Verify IND2 Values
data.IND2.value_counts()

SocialServices                30052
NIU                           28311
Manufacturing                 15500
ProfessinalServices           15350
RetailTrade                   13211
Construction                  10709
Entertainment                 10012
Finance                        9379
PublicAdmin                    6822
Other                          6161
TransportationWharehousing     5731
WholesaleTrade                 3766
InformationCommunications      3115
Agriculture                    2066
Utilities                      1214
ArmedForces                      21
Name: IND2, dtype: int64

Variable: OCC (Occupation)

In [66]:
# Recode data codes with labels
data.loc[data.OCC <= 420,'OCC2'] = 'Management, Business, Science, and Arts Occupations'
data.loc[(data.OCC <= 740) & (data.OCC > 420),'OCC2'] = 'Business Operations Specialists'
data.loc[(data.OCC <= 950) & (data.OCC > 740),'OCC2'] = 'Financial Specialists'
data.loc[(data.OCC <= 1240) & (data.OCC > 950),'OCC2'] = 'Computer and Mathematical Occupations'
data.loc[(data.OCC <= 1560) & (data.OCC > 1240),'OCC2'] = 'Architecture and Engineering Occupations'
data.loc[(data.OCC <= 1965) & (data.OCC > 1560),'OCC2'] = 'Life, Physical, and Social Science Occupations'
data.loc[(data.OCC <= 2060) & (data.OCC > 1965),'OCC2'] = 'Community and Social Services Occupations'
data.loc[(data.OCC <= 2145) & (data.OCC > 2060),'OCC2'] = 'Legal Occupations'
data.loc[(data.OCC <= 2550) & (data.OCC > 2145),'OCC2'] = 'Education, Training, and Library Occupations'
data.loc[(data.OCC <= 2920) & (data.OCC > 2550),'OCC2'] = 'Arts, Design, Entertainment, Sports, and Media Occupations'
data.loc[(data.OCC <= 3540) & (data.OCC > 2920),'OCC2'] = 'Healthcare Practitioners and Technical Occupations'
data.loc[(data.OCC <= 3655) & (data.OCC > 3540),'OCC2'] = 'Healthcare Support Occupations'
data.loc[(data.OCC <= 3955) & (data.OCC > 3655),'OCC2'] = 'Protective Service Occupations'
data.loc[(data.OCC <= 4150) & (data.OCC > 3955),'OCC2'] = 'Food Preparation and Serving Occupations'
data.loc[(data.OCC <= 4250) & (data.OCC > 4150),'OCC2'] = 'Building and Grounds Cleaning and Maintenance Occupations'
data.loc[(data.OCC <= 4650) & (data.OCC > 4250),'OCC2'] = 'Personal Care and Service Occupations'
data.loc[(data.OCC <= 4965) & (data.OCC > 4650),'OCC2'] = 'Sales and Related Occupations'
data.loc[(data.OCC <= 5940) & (data.OCC > 4965),'OCC2'] = 'Office and Administrative Support Occupations'
data.loc[(data.OCC <= 6130) & (data.OCC > 5940),'OCC2'] = 'Farming, Fishing, and Forestry Occupations'
data.loc[(data.OCC <= 6765) & (data.OCC > 6130),'OCC2'] = 'Construction and Extraction Occupations'
data.loc[(data.OCC <= 6940) & (data.OCC > 6765),'OCC2'] = 'Extraction Workers'
data.loc[(data.OCC <= 7610) & (data.OCC > 6940),'OCC2'] = 'Installation, Maintenance, and Repair Workers'
data.loc[(data.OCC <= 8950) & (data.OCC > 7610),'OCC2'] = 'Production Occupations'
data.loc[(data.OCC <= 9750) & (data.OCC > 8950),'OCC2'] = 'Transportation and Material Moving Occupations'
data.loc[(data.OCC <= 9920) & (data.OCC > 9750),'OCC2'] = 'Military Specific Occupations'
data.loc[data.OCC == 430,'OCC2'] = 'Management, Business, Science, and Arts Occupations'
data.loc[data.OCC == 726,'OCC2'] = 'Sales and Related Occupations'
data.loc[data.OCC == 9050,'OCC2'] = 'Personal Care and Service Occupations'
data.loc[data.OCC == 9415,'OCC2'] = 'Personal Care and Service Occupations'
# Verify DIFFANY2 Values
data.OCC2.value_counts()

Management, Business, Science, and Arts Occupations           43872
Office and Administrative Support Occupations                 16140
Sales and Related Occupations                                 12927
Education, Training, and Library Occupations                   8603
Transportation and Material Moving Occupations                 8509
Construction and Extraction Occupations                        8358
Healthcare Practitioners and Technical Occupations             7749
Production Occupations                                         7564
Food Preparation and Serving Occupations                       5661
Building and Grounds Cleaning and Maintenance Occupations      4830
Installation, Maintenance, and Repair Workers                  4680
Personal Care and Service Occupations                          4490
Computer and Mathematical Occupations                          3954
Business Operations Specialists                                3316
Financial Specialists                           

In [25]:
# Run a chi-squared test to see which variables are significantly related to employment status
data2 = data.drop(['EMPSTAT2'], axis=1)
for column in data2:
    variable_empstat=pd.crosstab(index=data['EMPSTAT2'],columns = data[column] ,colnames=[column])
   # print(variable_empstat)
   # print ("\n")
    stat, p, dof, expected = stats.chi2_contingency(variable_empstat)
    critical = stats.chi2.ppf(0.95, dof)
    print(column)
    print(0.95, critical, stat)
    if abs(stat) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    print('_________________________________________________')

YEAR
0.95 18.307038053275146 848.4213580213875
Dependent (reject H0)
_________________________________________________
SERIAL
0.95 310138.8800634387 335685.023329885
Dependent (reject H0)
_________________________________________________
MONTH
0.95 nan 0.0
Independent (fail to reject H0)
_________________________________________________
HWTFINL
0.95 281728.1192688053 332884.4203439012
Dependent (reject H0)
_________________________________________________
CPSID
0.95 507380.37985995814 553385.2103241322
Dependent (reject H0)
_________________________________________________
REGION
0.95 55.75847927888702 1425.3451754982625
Dependent (reject H0)
_________________________________________________
STATEFIP
0.95 287.8815005218308 3507.093109038604
Dependent (reject H0)
_________________________________________________
METRO
0.95 31.410432844230918 832.1225472007814
Dependent (reject H0)
_________________________________________________
COUNTY
0.95 1940.9057864615093 5982.426993548565
Dependen

In [67]:
# Keep the variables that have a significant correlation with EMPSTAT (reject H0)
data = data[['YEAR','FAMINC','AGE','OCC2','IND2','RACE2','SEX2','MARST2','YRIMMIG2','CITIZEN2','EDUC2','STATEFIP2','METRO2','VETSTAT2','EMPSTAT2']]

# Verify the dataset
data.head()

,YEAR,FAMINC,AGE,OCC2,IND2,RACE2,SEX2,MARST2,YRIMMIG2,CITIZEN2,EDUC2,STATEFIP2,METRO2,VETSTAT2,EMPSTAT2
0,2006,600,35,Office and Administrative Support Occupations,Finance,White,Female,Single,NIU,Born in US,High School Diploma or Equivalent,Alabama,Not in metro area,No service,Employed
2,2006,500,50,Business Operations Specialists,Finance,White,Female,"Seprated, divorced, or widowed",NIU,Born in US,Some College but No Degree,Alabama,Not in metro area,No service,Employed
6,2006,470,37,Office and Administrative Support Occupations,TransportationWharehousing,Black,Female,"Seprated, divorced, or widowed",NIU,Born in US,High School Diploma or Equivalent,Alabama,Not in metro area,No service,Employed
10,2006,843,33,Sales and Related Occupations,WholesaleTrade,White,Female,Married,NIU,Born in US,Bachelors Degree,Alabama,Not in metro area,No service,Employed
11,2006,843,34,Legal Occupations,ProfessinalServices,White,Male,Married,NIU,Born in US,Graduate Degree,Alabama,Not in metro area,No service,Employed


## Data Transformation

In [68]:
#Establish the list of columns we want to encode
Vars_to_transform = ['STATEFIP2',
                     'METRO2',
                     'FAMINC',
                      'AGE',
                      'OCC2',
                      'IND2',
                      'YRIMMIG2',
                      'RACE2',
                      'SEX2',
                      'MARST2',
                      'CITIZEN2',
                      'EDUC2',
                      'VETSTAT2',
                    'EMPSTAT2']

In [69]:
#Transform columns (col by col) and overwrite the existing columns, then save the dictionary in encoders
encoders = {}
data_enc = pd.DataFrame()
for D in Vars_to_transform:
    le = LabelEncoder()
    data_enc[D] = le.fit_transform(data[D])
    encoders[D] = le
    print("I'm transforming {}".format(D))
data_enc.head()

I'm transforming STATEFIP2
I'm transforming METRO2
I'm transforming FAMINC
I'm transforming AGE
I'm transforming OCC2
I'm transforming IND2


TypeError: '<' not supported between instances of 'str' and 'float'

In [ ]:
#Join the initial column Year with the encoded data
data_enc2 = data_enc.join(data['YEAR'], how = 'left')
data_enc2.head()

In [ ]:
# Split into three different datasets by year
data2016 = data_enc2[data_enc2['YEAR'] == 2016]
data2011 = data_enc2[data_enc2['YEAR'] == 2011]
data2006 = data_enc2[data_enc2['YEAR'] == 2006]

# Drop the year variable
data2016 = data2016.drop(['YEAR'], axis=1)
data2011 = data2011.drop(['YEAR'], axis=1)
data2006 = data2006.drop(['YEAR'], axis=1)

# Verify the datasets
print(data2006.head())
print(data2011.head())
print(data2016.head())

# Save the data to file

In [ ]:
# Writing cleaned data to four separate files
data_2016 = data2016.to_csv('data2016.csv', index = None, header = True)
data_2011 = data2011.to_csv('data2011.csv', index = None, header = True)
data_2006 = data2006.to_csv('data2006.csv', index = None, header = True)
data_enc2 = data.to_csv('IPUMS_CPS_Clean.csv', index = None, header=True)

In [29]:
# Create dateset for unemployed (need to decide whether we want this still)
data2006unemp = data2006[data2006['EMPSTAT2'] == "Unemployed"]
data2006unemp = data.to_csv('data2006unemp.csv', index = None, header=True)

data2011unemp = data2011[data2011['EMPSTAT2'] == "Unemployed"]
data2011unemp = data.to_csv('data2011unemp.csv', index = None, header=True)

data2016unemp = data2016[data2016['EMPSTAT2'] == "Unemployed"]
data2016unemp = data.to_csv('data2016unemp.csv', index = None, header=True)

data2006unemp = data2006unemp.to_csv('data2006unemp.csv', index = None, header = True)
data2011unemp = data2011unemp.to_csv('data2011unemp.csv', index = None, header = True)
data2016unemp = data2016unemp.to_csv('data2016unemp.csv', index = None, header = True)